In [ ]:
#library
import pandas as pd
import numpy as np

In [ ]:
#load data
file_path='/content/bank-additional-full.csv'
df= pd.read_csv(file_path, sep=';')
pd.set_option('display.max_columns', None)
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [ ]:
#Hapus kolom duration untuk mengatasi data leakage
df_processed = df.copy()
if 'duration' in df_processed.columns:
    df_processed.drop('duration', axis=1, inplace=True)
    print("\nKolom 'duration' telah dihapus.")
else:
    print("\nKolom 'duration' tidak ditemukan dalam dataset.")


Kolom 'duration' telah dihapus.


In [ ]:
#penanganan fitur pdays
# Membuat fitur baru 'pernah_dihubungi' yang informatif.
if 'pdays' in df_processed.columns:
    df_processed['pernah_dihubungi'] = (df_processed['pdays'] != 999).astype(int)
    df_processed.drop('pdays', axis=1, inplace=True)
    print("Fitur 'pdays' telah diubah.")
else:
    print("Kolom 'pdays' tidak ditemukan dalam dataset.")

Fitur 'pdays' telah diubah.


In [ ]:
# imputasi unknown
# Justifikasi: Kami memilih modus karena ini adalah metode imputasi yang sederhana dan cepat.
# Asumsinya adalah nilai yang tidak diketahui kemungkinan besar mengikuti distribusi
# Cek kolom yang ada dan mengandung 'unknown'
possible_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan']
columns_to_impute = []

for col in possible_columns:
    if col in df_processed.columns:
        if df_processed[col].dtype == 'object' and 'unknown' in df_processed[col].values:
            columns_to_impute.append(col)

if columns_to_impute:
    for column in columns_to_impute:
        mode_value = df_processed[column].mode()[0]
        count_unknown = (df_processed[column] == 'unknown').sum()
        df_processed[column] = df_processed[column].replace('unknown', mode_value)
        print(f"{column}: {count_unknown} nilai 'unknown' → '{mode_value}'")
    print(f"Nilai 'unknown' di kolom {columns_to_impute} telah diisi dengan modus.")
else:
    print("Tidak ada nilai 'unknown' yang perlu diimputasi.")

job: 330 nilai 'unknown' → 'admin.'
marital: 80 nilai 'unknown' → 'married'
education: 1731 nilai 'unknown' → 'university.degree'
default: 8597 nilai 'unknown' → 'no'
housing: 990 nilai 'unknown' → 'yes'
loan: 990 nilai 'unknown' → 'no'
Nilai 'unknown' di kolom ['job', 'marital', 'education', 'default', 'housing', 'loan'] telah diisi dengan modus.


In [ ]:
#encoding
# Cari nama kolom target (bisa 'y' atau 'deposit')
target_column = None
if 'y' in df_processed.columns:
    target_column = 'y'
elif 'deposit' in df_processed.columns:
    target_column = 'deposit'
else:
    print("ERROR: Kolom target tidak ditemukan!")
    print(f"Kolom yang tersedia: {df_processed.columns.tolist()}")
    raise KeyError("Target column not found")

print(f"Target column: '{target_column}'")

# Pisahkan fitur (X) dan target (y) SEBELUM encoding.
X = df_processed.drop(target_column, axis=1)
y = df_processed[target_column]

print(f"  Shape X: {X.shape}")
print(f"  Shape y: {y.shape}")

Target column: 'y'
  Shape X: (41188, 19)
  Shape y: (41188,)


In [ ]:
# Lakukan mapping pada target secara eksplisit
print(f"\nDistribusi target sebelum encoding:")
print(y.value_counts())

y = y.map({'yes': 1, 'no': 0})
print(f"\nVariabel target '{target_column}' telah di-map ke format biner (1=yes, 0=no).")
print(f"Distribusi target setelah encoding:")
print(y.value_counts())



Distribusi target sebelum encoding:
y
no     36548
yes     4640
Name: count, dtype: int64

Variabel target 'y' telah di-map ke format biner (1=yes, 0=no).
Distribusi target setelah encoding:
y
0    36548
1     4640
Name: count, dtype: int64


In [ ]:
# Lakukan One-Hot Encoding hanyapada fitur (X)
categorical_cols = X.select_dtypes(include=['object']).columns
print(f"\nKolom kategorikal yang akan di-encode ({len(categorical_cols)}): {list(categorical_cols)}")

X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
print(f"  Fitur kategorikal telah di-encode menggunakan One-Hot Encoding.")
print(f"  Jumlah fitur sebelum encoding: {X.shape[1]}")
print(f"  Jumlah fitur setelah encoding: {X_encoded.shape[1]}")


Kolom kategorikal yang akan di-encode (10): ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
  Fitur kategorikal telah di-encode menggunakan One-Hot Encoding.
  Jumlah fitur sebelum encoding: 19
  Jumlah fitur setelah encoding: 46


In [ ]:
# Menggabungkan kembali X yang sudah di-encode dan y untuk analisis atau penyimpanan
df_final = pd.concat([X_encoded, y], axis=1)
print(f"Bentuk data akhir: {df_final.shape}")
print(f"  - Total baris: {df_final.shape[0]:,}")
print(f"  - Total fitur: {df_final.shape[1] - 1}")
print(f"  - Target column: '{target_column}' → 'y' (encoded)")

# Tampilkan beberapa baris pertama dari data yang sudah benar-benar bersih
print("\n Preview data final (5 baris pertama):")
df_final.head()



Bentuk data akhir: (41188, 47)
  - Total baris: 41,188
  - Total fitur: 46
  - Target column: 'y' → 'y' (encoded)

 Preview data final (5 baris pertama):


,age,campaign,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,pernah_dihubungi,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_married,marital_single,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,default_yes,housing_yes,loan_yes,contact_telephone,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success,y
0,56,1,0,1.1,93.994,-36.4,4.857,5191.0,0,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,0
1,57,1,0,1.1,93.994,-36.4,4.857,5191.0,0,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,0
2,37,1,0,1.1,93.994,-36.4,4.857,5191.0,0,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,0
3,40,1,0,1.1,93.994,-36.4,4.857,5191.0,0,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,0
4,56,1,0,1.1,93.994,-36.4,4.857,5191.0,0,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,True,True,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,0


In [ ]:
#save data
if target_column != 'y':
    df_final = df_final.rename(columns={target_column: 'y'})
    print(f"Kolom target di-rename dari '{target_column}' → 'y'")

# Save to CSV
output_path = "bank_additional_cleaned_v2.csv"

try:
    df_final.to_csv(output_path, index=False)
    print(f"Data berhasil disimpan di: {output_path}")
except Exception as e:
    print(f"Gagal menyimpan data: {e}")

Data berhasil disimpan di: bank_additional_cleaned_v2.csv
